In [ ]:
import backtrader as bt
import pandas as pd
import os

# --------------------------
# 核心配置（不变）
# --------------------------
CONFIG = {
    "selection_path": r'D:\workspace\xiaoyao\works\trytry\上涨股票的筛选探索1024\short_term_selection_optimized.csv',
    "data_path": r'D:\workspace\xiaoyao\data\widetable.parquet',
    "initial_cash": 1000000,
    "buy_delay": 1,
    "hold_days": 5,
    "commission_rate": 0.0003,
    "stamp_tax_rate": 0.001,
    "slippage_rate": 0.001,
    "max_single_pos_ratio": 0.2,  # 单票20%仓位
}

# --------------------------
# 工具函数（不变）
# --------------------------
def log(msg):
    timestamp = pd.Timestamp.now().strftime('%H:%M:%S')
    print(f"[{timestamp}] {msg}")

def load_target_data():
    # 1. 读CSV选股权重
    log("加载选股结果CSV（T日数据）...")
    csv_df = pd.read_csv(CONFIG["selection_path"])
    csv_df['date'] = pd.to_datetime(csv_df['date']).dt.date
    csv_df = csv_df.dropna(subset=['stock_code', 'date'])
    required_codes = list(csv_df['stock_code'].unique())
    selection_by_date = dict(csv_df.groupby('date')['stock_code'].apply(list))
    log(f"CSV筛选结果：{len(selection_by_date)}个选股日，涉及{len(required_codes)}只股票")
    if not required_codes:
        raise ValueError("选股结果中无有效股票代码")

    # 2. 读宽表（仅需的股票）
    log(f"加载宽表数据（仅{len(required_codes)}只股票）...")
    wide_df = pd.read_parquet(CONFIG["data_path"])
    wide_df['date'] = pd.to_datetime(wide_df['date']).dt.date
    wide_df = wide_df[wide_df['stock_code'].isin(required_codes)][['stock_code', 'date', 'open', 'close']].dropna()
    if wide_df.empty:
        raise ValueError("宽表无目标股票数据")

    # 3. 价格映射和交易日列表
    price_map = {(row['stock_code'], row['date']): {'open': row['open'], 'close': row['close']} 
                 for _, row in wide_df.iterrows()}
    log(f"价格映射覆盖{len(price_map)}条（股票-日期）记录")
    market_trade_dates = sorted(wide_df['date'].unique())
    date_to_idx = {date: idx for idx, date in enumerate(market_trade_dates)}
    if not market_trade_dates:
        raise ValueError("无有效交易日数据")

    # 4. 时间轴数据（驱动回测日期）
    time_axis_data = None
    for code in required_codes:
        sample_df = wide_df[wide_df['stock_code'] == code].sort_values('date')
        if not sample_df.empty:
            sample_df['datetime'] = pd.to_datetime(sample_df['date'])
            time_axis_data = bt.feeds.PandasData(
                dataname=sample_df,
                datetime='datetime',
                open='open',
                high='open',  # 填充high/low避免报错
                low='open',
                close='close',
                volume=None,
                openinterest=None
            )
            time_axis_data._name = "time_axis"
            log(f"用{code}作为时间轴（{len(sample_df)}个交易日）")
            break

    if time_axis_data is None:
        raise ValueError("所有股票都没有足够数据构建时间轴")

    return selection_by_date, price_map, market_trade_dates, date_to_idx, time_axis_data

# --------------------------
# 策略部分（核心：改了持仓变量名）
# --------------------------
class FixedTradeStrategy(bt.Strategy):
    params = (
        ("selection_by_date", None),
        ("price_map", None),
        ("market_trade_dates", None),
        ("date_to_idx", None),
        ("buy_delay", CONFIG["buy_delay"]),
        ("hold_days", CONFIG["hold_days"]),
        ("max_single_pos_ratio", CONFIG["max_single_pos_ratio"]),
    )

    def __init__(self):
        self.buy_plans = {}
        # 关键修改：把self.positions改成self.stock_positions，避免和内置属性冲突
        self.stock_positions = {}  # 格式：{股票代码: {'shares': ..., 'buy_price': ..., 'buy_date': ...}}
        self.price_map = self.p.price_map
        self.date_to_idx = self.p.date_to_idx
        self.trade_dates = self.p.market_trade_dates

    def next(self):
        today = self.datas[0].datetime.date()  # 当前交易日

        # 1. 生成买入计划（T日选股→T+1买入）
        if today in self.p.selection_by_date:
            try:
                t_idx = self.date_to_idx[today]
                buy_idx = t_idx + self.p.buy_delay
                if buy_idx >= len(self.trade_dates):
                    raise IndexError
                buy_date = self.trade_dates[buy_idx]
            except (KeyError, IndexError):
                log(f"警告：{today}的T+1买入日无效，跳过")
                return
            self.buy_plans[buy_date] = self.p.selection_by_date[today]
            log(f"{today}选股{len(self.p.selection_by_date[today])}只，计划{buy_date}买入（单票≤20%总资产）")

        # 2. 执行买入（单票最大20%总资产）
        if today in self.buy_plans:
            target_stocks = self.buy_plans[today]
            total_asset = self.broker.get_value()  # 总资产=现金+持仓市值
            available_cash = self.broker.get_cash()
            max_cash_per_stock = total_asset * self.p.max_single_pos_ratio  # 单票20%总资产

            for code in target_stocks:
                price_key = (code, today)
                if price_key not in self.price_map:
                    log(f"警告：{code}在{today}无价格，跳过")
                    continue
                buy_open = self.price_map[price_key]['open']
                buy_price = buy_open * (1 + CONFIG["slippage_rate"])
                if buy_price <= 0:
                    log(f"警告：{code}买入价异常，跳过")
                    continue

                max_shares = int(max_cash_per_stock / buy_price / 100) * 100
                if max_shares <= 0:
                    log(f"警告：{code}资金不足（单票最多可买{max_cash_per_stock:.2f}元），跳过")
                    continue

                buy_cost = buy_price * max_shares
                if buy_cost > available_cash:
                    log(f"警告：{code}可用资金不足（需{buy_cost:.2f}，可用{available_cash:.2f}），跳过")
                    continue

                self.buy(data=self.datas[0], size=max_shares, price=buy_price)
                # 关键修改：用self.stock_positions记录持仓
                self.stock_positions[code] = {
                    'shares': max_shares,
                    'buy_price': buy_price,
                    'buy_date': today
                }
                available_cash -= buy_cost
                log(f"买入 {code}：{buy_price:.2f}元 × {max_shares}股，成本{buy_cost:.2f}元（占总资产{buy_cost/total_asset*100:.1f}%）")

            del self.buy_plans[today]

        # 3. 执行卖出（持有满5天）
        # 关键修改：遍历self.stock_positions
        for code in list(self.stock_positions.keys()):
            pos_info = self.stock_positions[code]
            buy_date = pos_info['buy_date']
            try:
                buy_idx = self.date_to_idx[buy_date]
                today_idx = self.date_to_idx[today]
                hold_days = today_idx - buy_idx
            except KeyError:
                hold_days = self.p.hold_days + 1

            if hold_days >= self.p.hold_days:
                price_key = (code, today)
                if price_key not in self.price_map:
                    log(f"警告：{code}在{today}无价格，强制清仓")
                    del self.stock_positions[code]
                    continue
                sell_close = self.price_map[price_key]['close']
                sell_price = sell_close * (1 - CONFIG["slippage_rate"])
                if sell_price <= 0:
                    log(f"警告：{code}卖出价异常，强制清仓")
                    del self.stock_positions[code]
                    continue

                sell_shares = pos_info['shares']
                buy_amount = pos_info['buy_price'] * sell_shares
                sell_amount = sell_price * sell_shares
                commission = (buy_amount + sell_amount) * CONFIG["commission_rate"]
                stamp_tax = sell_amount * CONFIG["stamp_tax_rate"]
                total_cost = commission + stamp_tax
                profit = sell_amount - buy_amount - total_cost
                profit_rate = (profit / buy_amount) * 100 if buy_amount != 0 else 0

                self.sell(data=self.datas[0], size=sell_shares, price=sell_price)
                self.broker.setcash(self.broker.get_cash() - total_cost)
                del self.stock_positions[code]  # 关键修改：删除持仓记录

                log(f"卖出 {code}：持有{hold_days}天，{sell_price:.2f}元 × {sell_shares}股，收益{profit:.2f}元（{profit_rate:.2f}%），成本{total_cost:.2f}元")

    def stop(self):
        # 强制清仓剩余持仓
        # 关键修改：遍历self.stock_positions
        for code in list(self.stock_positions.keys()):
            pos_info = self.stock_positions[code]
            log(f"回测结束，强制清仓{code}：{pos_info['shares']}股")
            del self.stock_positions[code]

        final_asset = self.broker.get_value()
        total_profit = final_asset - CONFIG["initial_cash"]
        total_return = (total_profit / CONFIG["initial_cash"]) * 100
        log(f"\n{'='*60}")
        log(f"回测结束：初始{CONFIG['initial_cash']:,}元 → 最终{final_asset:,.2f}元")
        log(f"总收益：{total_profit:,.2f}元（总收益率 {total_return:.2f}%）")
        log(f"{'='*60}")

# --------------------------
# 主函数（不变）
# --------------------------
def run_backtest():
    try:
        selection_by_date, price_map, market_trade_dates, date_to_idx, time_axis_data = load_target_data()
        cerebro = bt.Cerebro()
        cerebro.broker.setcash(CONFIG["initial_cash"])
        cerebro.broker.set_slippage_perc(perc=0)
        cerebro.broker.setcommission(commission=0)

        cerebro.adddata(time_axis_data)
        cerebro.addstrategy(
            FixedTradeStrategy,
            selection_by_date=selection_by_date,
            price_map=price_map,
            market_trade_dates=market_trade_dates,
            date_to_idx=date_to_idx
        )

        log(f"\n开始回测：{market_trade_dates[0]}至{market_trade_dates[-1]}（单票最大20%仓位）")
        cerebro.run()

    except Exception as e:
        log(f"回测失败：{str(e)}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    run_backtest()

[16:07:08] 加载选股结果CSV（T日数据）...
[16:07:08] CSV筛选结果：253个选股日，涉及548只股票
[16:07:08] 加载宽表数据（仅548只股票）...
[16:07:39] 价格映射覆盖370470条（股票-日期）记录
[16:07:39] 用002298.XSHE作为时间轴（678个交易日）
[16:07:39] 
开始回测：2023-01-03至2025-10-23（单票最大20%仓位）
[16:07:39] 回测失败：can't set attribute 'positions'


Traceback (most recent call last):
  File "C:\Users\jay\AppData\Local\Temp\ipykernel_32700\989744825.py", line 235, in run_backtest
    cerebro.run()
  File "d:\sdk\Anaconda3\envs\xiaoyao\lib\site-packages\backtrader\cerebro.py", line 1132, in run
    runstrat = self.runstrategies(iterstrat)
  File "d:\sdk\Anaconda3\envs\xiaoyao\lib\site-packages\backtrader\cerebro.py", line 1222, in runstrategies
    strat = stratcls(*sargs, **skwargs)
  File "d:\sdk\Anaconda3\envs\xiaoyao\lib\site-packages\backtrader\metabase.py", line 88, in __call__
    _obj, args, kwargs = cls.doinit(_obj, *args, **kwargs)
  File "d:\sdk\Anaconda3\envs\xiaoyao\lib\site-packages\backtrader\metabase.py", line 78, in doinit
    _obj.__init__(*args, **kwargs)
  File "C:\Users\jay\AppData\Local\Temp\ipykernel_32700\989744825.py", line 99, in __init__
    self.positions = {}  # 手动记录持仓：{股票代码: {'shares': ..., 'buy_price': ..., 'buy_date': ...}}
AttributeError: can't set attribute 'positions'
